In [1]:
# For each encoding, takes the paratope-epitope file (two-columns) and cluster the encoded paratopes by their LD. 
# The epitopes are used for coloring.
# HDBscan estimates boundary between clusters and give them an ID.
# Two output figures: paratope clustering colored by epitope; paratope clustering colored by HBDscan.
# The input files are balanced by groups of 500 paratopes for 1 epitope

In [2]:
import jellyfish
import os
import csv
import sys
import numpy as np
# this script seems only to work with version: pip install numpy==1.20.0

In [3]:
print(np.__version__)

1.20.0


In [4]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score as ads
from sklearn.metrics import normalized_mutual_info_score as nis
from sklearn.metrics import mutual_info_score as mis
from random import randint

import plotly.express as px
import Levenshtein as lv
import scipy.spatial as sp, scipy.cluster.hierarchy as hc
import scipy
from scipy.cluster.hierarchy import ward, fcluster

from plotly.graph_objs import *

In [5]:
np.random.seed(42)

In [6]:
#pip install umap.learn
import umap.umap_ as umap
import seaborn as sns; sns.set()
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [7]:
import hdbscan
import plotly.subplots as sp

In [8]:
maxLines = 100000000

In [9]:
def readParaEpiFile(fileName):
    epiParaSeq = open(fileName, newline = '')   #one line is a text with \t and \n                                                                              
    data_reader = csv.reader(epiParaSeq, delimiter='\t') #transform lines into lists 
    paratopes = []
    epitopes = []
    paraepi = {}
    cpt = 0
    for line in data_reader:
        if((cpt > 0) and (cpt <= maxLines)): 
            paratopes.append(line[1])
            epitopes.append(line[0])
            if line[0] in paraepi:
                paraepi[line[0]].append(line[1])
            else:
                paraepi[line[0]] = [line[1]] 
        cpt = cpt + 1

    print("Got ", len(paratopes), " paratopes and ", len(epitopes), " epitopes from ", fileName)
    return (paratopes, epitopes, paraepi)

In [22]:
folder = ""
folderOutput = ""

listFiles = ["Task4_A_EpiSeq_ParaSeq.txt_Balanced_500.txt",
"Task4_A_EpiSeq_ParaSeqD2.txt_Balanced_500.txt",
"Task4_B_EpiMotif_ParaMotifD2.txt_Balanced_100.txt",
"Task4_C_EpiAgr_ParaAgr.txt_Balanced_500.txt",
"Task4_C_EpiAgr_ParaAgrD2.txt_Balanced_500.txt",
"Task4_D_EpiChem_ParaChem.txt_Balanced_500.txt",
"Task4_D_EpiChem_ParaChemD2.txt_Balanced_500.txt",
"Task4_E_EpiSeq_ParaSeq_NoDeg.txt_Balanced_500.txt",
"Task4_E_EpiSeq_ParaSeq_NoDegD2.txt_Balanced_500.txt",
"Task4_F_EpiMotif_ParaMotif_NoDegD2.txt_Balanced_100.txt",
"Task4_G_EpiAgr_ParaAgr_NoDeg.txt_Balanced_500.txt",
"Task4_G_EpiAgr_ParaAgr_NoDegD2.txt_Balanced_500.txt",
"Task4_H_EpiChem_ParaChem_NoDeg.txt_Balanced_500.txt",
"Task4_H_EpiChem_ParaChem_NoDegD2.txt_Balanced_500.txt"]
nFiles = len(listFiles)
print(nFiles)

listOutputPictures = ["Task4_A_EpiSeq_ParaSeq",
"Task4_A_EpiSeq_ParaSeqD2",
"Task4_B_EpiMotif_ParaMotifD2",
"Task4_C_EpiAgr_ParaAgr",
"Task4_C_EpiAgr_ParaAgrD2",
"Task4_D_EpiChem_ParaChem",
"Task4_D_EpiChem_ParaChemD2",
"Task4_E_EpiSeq_ParaSeq_NoDeg",
"Task4_E_EpiSeq_ParaSeq_NoDegD2",
"Task4_F_EpiMotif_ParaMotif_NoDegD2",
"Task4_G_EpiAgr_ParaAgr_NoDeg",
"Task4_G_EpiAgr_ParaAgr_NoDegD2",
"Task4_H_EpiChem_ParaChem_NoDeg",
"Task4_H_EpiChem_ParaChem_NoDegD2"]

14


In [17]:
analysisDim = 10000

col_list = sns.color_palette("Spectral", 20)
my_palette = col_list.as_hex()
print(my_palette)
my_opacity = 0.5

layoutNoLegend = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    width=800, height=800
)

layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=True,
    width=800, height=800
)

['#b81e48', '#d23a4e', '#e2514a', '#f06744', '#f7844e', '#fca55d', '#fdbf6f', '#fed683', '#fee999', '#fff7b2', '#f9fcb5', '#edf8a3', '#daf09a', '#bfe5a0', '#a2d9a4', '#7ecca5', '#60bba8', '#47a0b3', '#3585bb', '#496aaf']


In [23]:
for i in [2,9]:
    fileIn = folder + listFiles[i]
    fileFig = folderOutput + listOutputPictures[i]
    
    [para, epi, paraepi] = readParaEpiFile(fileIn)
    
    setPara = para #We keep the order
    setEpi = set(epi)
    print("Got ", len(setPara), " unique paratopes and ", len(setEpi), " unique epitopes")
    
    ab_sequence = list(setPara[0:analysisDim])
    distmat = np.zeros((len(ab_sequence), len(ab_sequence)))
    distmat_norm = np.zeros((len(ab_sequence), len(ab_sequence)))

    for i in range(len(ab_sequence)):
        if i%500 == 0:
            print(i)
        for j in range(len(ab_sequence)):
            distmat[i,j] = lv.distance(ab_sequence[i],ab_sequence[j]) 
            distmat_norm[i,j] = distmat[i,j] / (max(1, max(len(ab_sequence[i]), len(ab_sequence[j]))))
    
    labels = epi[0:analysisDim]
    
    U = umap.UMAP(metric='precomputed', n_neighbors=30)
    XY = U.fit_transform(distmat_norm)

    fig1 = px.scatter(XY, x=0, y=1, color=labels, labels=dict(x="X axis", y="Y axis", labels="Epitope"), hover_data=[ab_sequence], color_discrete_sequence = my_palette, opacity = my_opacity)
    fig1.layout = layoutNoLegend    
    fig1.update_yaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
    fig1.update_xaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
    fig1.show()
    fig1.write_image(fileFig + "ClassesNoLeg" + ".png")
    fig1.write_image(fileFig + "ClassesNoLeg" + ".svg")
    
    fig1.layout = layout
    fig1.update_yaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
    fig1.update_xaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
    fig1.show()
    fig1.write_image(fileFig + "Classes" + ".png")
    fig1.write_image(fileFig + "Classes" + ".svg")

    
    labels2 = hdbscan.HDBSCAN(
        min_samples=50,
        min_cluster_size=50,
    ).fit_predict(XY)
    
    labels2 = [str(x) for x in labels2]
    
    fig2 = px.scatter(XY, x=0, y=1, color=labels2, labels=dict(x="X axis", y="Y axis", labels="Epitope"), hover_data=[ab_sequence], color_discrete_sequence = my_palette, opacity = my_opacity)
    fig2.layout = layoutNoLegend    
    fig2.update_yaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
    fig2.update_xaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
    fig2.show()
    fig2.write_image(fileFig + "HdbscanNoLeg" + ".png")
    fig2.write_image(fileFig + "HdbscanNoLeg" + ".svg")
    
    fig2.layout = layout
    fig2.update_yaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
    fig2.update_xaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
    fig2.show()
    fig2.write_image(fileFig + "Hdbscan" + ".png")
    fig2.write_image(fileFig + "Hdbscan" + ".svg")

Got  1800  paratopes and  1800  epitopes from  Task4_B_EpiMotif_ParaMotifD2.txt_Balanced_100.txt
Got  1800  unique paratopes and  18  unique epitopes
0
500
1000
1500


c:\python\lib\site-packages\umap\umap_.py:1735: UserWarning:

using precomputed metric; transform will be unavailable for new data and inverse_transform will be unavailable for all data



Got  900  paratopes and  900  epitopes from  Task4_F_EpiMotif_ParaMotif_NoDegD2.txt_Balanced_100.txt
Got  900  unique paratopes and  9  unique epitopes
0
500


c:\python\lib\site-packages\umap\umap_.py:1735: UserWarning:

using precomputed metric; transform will be unavailable for new data and inverse_transform will be unavailable for all data



In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig1 = px.scatter(XY, x=0, y=1, color=labels, labels=dict(x="X axis", y="Y axis", labels="Epitope"), hover_data=[ab_sequence], color_discrete_sequence = my_palette, opacity = my_opacity)
fig1.layout = layout
fig1.update_yaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
fig1.update_xaxes(visible=True, linecolor = 'black', linewidth = 1, tickmode = 'array', color = 'white')
fig1.show()
fig1.write_image("testNoLegend.png")